In [1]:
import numpy as np
from neurom.io import swc
from tifffile import imsave, imwrite
from scipy.spatial import distance
import glob
import re

def list_sphere(center, r):
    list_coords = []
    for x in range(-r,r+1):
        for y in range(-r,r+1):
            for z in range(-r,r+1):
                if distance.euclidean((center[0]+x,center[1]+y,center[2]+z),center) <= r:
                    list_coords.append((center[0]+x,center[1]+y,center[2]+z));
    return list_coords

def list_cube(center, r):
    list_coords = []
    for x in range(-r,r+1):
        for y in range(-r,r+1):
            for z in range(-r,r+1):
                list_coords.append((center[0]+x,center[1]+y,center[2]+z))
    return list_coords




In [6]:
#Load RGB information from color log
filename = "Z:/fred/dec2019/3rdBatch_PVcreSecondImaging/BlR/logChannelIntensity.txt"

r = []
g = []
b = []

with open(filename) as f:
    for line in f:
        rgb_data = line.split(', ')
        b.append(float(rgb_data[3]))
        g.append(float(rgb_data[2]))
        r.append(float(rgb_data[4][0:-2]))
r = np.array(r);
g = np.array(g);
b = np.array(b);
#normalize the RGB values
rgb_data = np.stack((r,g,b),axis = -1)
for i in range(0,len(rgb_data)):
    maxRGB = np.sum(rgb_data[i]);
    rgb_data[i,0] = 255*rgb_data[i,0]/maxRGB;
    rgb_data[i,1] = 255*rgb_data[i,1]/maxRGB;
    rgb_data[i,2] = 255*rgb_data[i,2]/maxRGB;
#code for random RGB values
#    rgb_data[i] = np.random.randint(0,255,(1,3),dtype = 'uint8')

##Code to load pre-randomized RGB values
#rgb_fn =  'E:/randRGBcolors.csv'
#with open(rgb_fn, 'r', encoding='utf-8-sig') as f: 
#    rgb_data = np.genfromtxt(f, dtype=int, delimiter=',')

rgb_data = rgb_data.astype(int)


axon_fn = 'Z:/fred/dec2019/3rdBatch_PVcreSecondImaging/BlR/listAxons.csv'
with open(axon_fn, 'r', encoding='utf-8-sig') as f: 
    axonIDs = np.genfromtxt(f, dtype=float, delimiter=',')


#Import folder with SWC files

files = glob.glob('Z:/fred/dec2019/3rdBatch_PVcreSecondImaging/BlR/swc/*.swc' )

#Set this to the image's dimensions
imgdata = np.zeros(shape=(511,2100,2100,3), dtype=np.uint8)  #Z,X,Y, CHannels
    
    
#Load swc data into variable totalData only if it's included in list of axons (AxonIDs)
tot_data = []
idx = []
for z, swc_fn in enumerate(files):
    name = swc_fn.split('\\')[1]
    name_num = int(re.findall('\d+', name)[-1])
    if name_num in axonIDs:    
        print(name_num)
        data = np.genfromtxt(swc_fn,comments = "#", delimiter = ' ')
        tot_data.append(data.astype(int))
        idx.append(name_num)

rgb_idx =  [x - 1 for x in idx];


416
400
51
18
162
336
111
345
192
320
353
107
184
399
369
384
120
1
374
392
311
145
362
136
358
199
427
431
60
90
207
292
365
316
395
6
373
127
154
383
349
188
67
81
71
420
295
216
265
25
56
278
407
40
411
227
354
327
342
331
165
378
388
322
351
105
334
347
8
318
402
414
36
62
433
425
290
409
276
58
205
313
390
360
134
386
122
376
3
329
422
73
16
434
65
49
418
371
4
125
381
302
156
367
397
314
338
340
333
167
356
171
325
138
309
413
42
405
54
429
339
370
5
380
366
396
141
315
203
280
419
82
423
435
79
257
412
26
404
55
139
308
341
332
103
324
52
403
415
128
9
319
323
350
335
346
119
328
312
391
361
304
387
123
377
2
277
408
59
432
93
10
75
424
272
66
248
421
348
189
179
364
317
394
7
372
126
382
379
389
182
101
355
172
326
343
330
406
279
32
41
410
398
368
337
344
321
175
352
417
401
426
430
61
91
359
385
152
306
375
393
363
137


In [7]:
#Iterate through list of neurons and draw swc as cube

for j in range(0,len(tot_data)):
    for i in range(0,len(tot_data[j])):
        center = (tot_data[j][i,2],tot_data[j][i,3],tot_data[j][i,4])
        neurite = tot_data[j][i,1];
        if neurite == 3 or tot_data[j][i,6] == -1: radius = 3;   #3 and -1 counts as dendrite and have thick lines(In nTracer set dendrites to 3). Everything else is axon. 
        else: radius = 1;
        list_coords = list_cube(center, radius);
        for ii in range(0,len(list_coords)):
            coords = (list_coords[ii][2],list_coords[ii][1],list_coords[ii][0]);
            if coords[0]< imgdata.shape[0] and coords[1]<imgdata.shape[1] and coords[2]< imgdata.shape[2]:
                if coords[0]>0 and coords[1]>0 and coords[2]>0:
                    imgdata[coords] = rgb_data[rgb_idx[j]];  #Z,X,Y
    print('neuron ' + str(rgb_idx[j]));


neuron 415
neuron 399
neuron 50
neuron 17
neuron 161
neuron 335
neuron 110
neuron 344
neuron 191
neuron 319
neuron 352
neuron 106
neuron 183
neuron 398
neuron 368
neuron 383
neuron 119
neuron 0
neuron 373
neuron 391
neuron 310
neuron 144
neuron 361
neuron 135
neuron 357
neuron 198
neuron 426
neuron 430
neuron 59
neuron 89
neuron 206
neuron 291
neuron 364
neuron 315
neuron 394
neuron 5
neuron 372
neuron 126
neuron 153
neuron 382
neuron 348
neuron 187
neuron 66
neuron 80
neuron 70
neuron 419
neuron 294
neuron 215
neuron 264
neuron 24
neuron 55
neuron 277
neuron 406
neuron 39
neuron 410
neuron 226
neuron 353
neuron 326
neuron 341
neuron 330
neuron 164
neuron 377
neuron 387
neuron 321
neuron 350
neuron 104
neuron 333
neuron 346
neuron 7
neuron 317
neuron 401
neuron 413
neuron 35
neuron 61
neuron 432
neuron 424
neuron 289
neuron 408
neuron 275
neuron 57
neuron 204
neuron 312
neuron 389
neuron 359
neuron 133
neuron 385
neuron 121
neuron 375
neuron 2
neuron 328
neuron 421
neuron 72
neuron 15


In [8]:
#Load list of Synapses in csv format (x,y,z)
syn_fn = 'Z:/fred/dec2019/3rdBatch_PVcreSecondImaging/BlR/listSyn.csv'
with open(syn_fn, 'r', encoding='utf-8-sig') as f: 
    synData = np.genfromtxt(f, dtype=float, delimiter=',')

#Draw syn ROIs as spheres
for i in range(0,len(synData)):
    center = (synData[i][0],synData[i][1],synData[i][2]);
    list_coords = list_sphere(center, 8);
    for ii in range(0,len(list_coords)):
            coords = (list_coords[ii][2],list_coords[ii][1],list_coords[ii][0]);
            coords = np.array(coords)
            if coords[0]< imgdata.shape[0] and coords[1]<imgdata.shape[1] and coords[2]< imgdata.shape[2]:
                if coords[0]>0 and coords[1]>0 and coords[2]>0:
                    synColor = np.array((255,255,255))   #Yellow is (255,255,0)
                    imgdata[tuple(coords.astype(int))] = synColor  #Z,X,Y

In [9]:
#Save file
outfn = 'E:/test.tif';
imsave(outfn, imgdata, metadata = {'DimensionOrder':'ZXYCT'});

In [16]:
# code to save RGB colors that were randomly generated
out_fn =  'E:/randRGBcolors.csv'
np.savetxt(out_fn,rgb_data, fmt = '%d', delimiter = ',')